New chip seq notebook: 
- All of the datasets downloaded in previous notebook lacked information
- I think that the excel files have information I need, which means I need to redo all of the work from the other notebook with downloaded excel files as pandas dataframes
- 166 chipseq d. mel datasets from modENCODE
- Only downloading 159 of those because modMINE lacks information for the rest (see no_data list)

In [1]:
import pandas as pd
import urllib
import os
from urllib.request import urlopen

In [2]:
url = 'http://data.modencode.org/cgi-bin/cloud_list.pl?accessions=3393,3806,3825,3231,2625,2626,2637,3403,4078,3240,4080,5068,4082,4081,3959,5069,2638,2639,3395,3235,4974,5008,5070,5071,5072,5577,3229,3230,3402,3401,2640,2641,2642,3234,3236,3239,3241,3400,3398,4976,3824,3826,4089,3809,3238,3397,5028,3814,3245,3830,4119,4981,5257,3815,4982,3827,2629,2630,2635,2636,2633,2634,2627,2628,2631,2632,3391,3392,4091,3956,4936,5592,5111,5110,5590,5587,5112,5591,5264,5113,5593,4944,3390,5023,4998,4070,3242,5114,5575,5115,5594,3243,3394,3812,5004,5005,5116,5597,5574,5117,4094,5118,854,858,3233,862,863,856,849,857,855,985,859,860,861,5119,4074,4107,3957,3237,3813,3816,5120,5121,5598,4095,4096,3960,3808,4098,5576,5570,5579,5571,984,850,848,4352,5580,5569,5122,5123,5124,846,847,851,852,853,5125,5126,5599,4099,4071,3232,5568,5014,4103,5017,4069,4104,3958,4716,4105,4943,5127,5606,3396,5128,5024,5025,3399,4114,4113,3823,5029,5129&urls=1'
#protein=',3954,4351,2755,3253,2753,2979,2754,3625,2783,3247,3251,4193'
page = urlopen(url).read().decode('utf-8')
page = page.split('\n')

modid_list = []
for line in page:
    if not line.startswith('#'):
        modid = str(line.split()[0]) #gives an error but still works? 
        modid_list.append(modid)

IndexError: list index out of range

In [3]:
len(set(modid_list))

188

In [4]:
#define 18? ids that modmine has no data for (unknown reasons...)
no_data = ['2635', '2625', '2637', '2631', '3806', '2632', '3827', '3826', '3235', '3236', '3823', '3824', '3825', '2640', '3231', '3397', '3233', '2628'] 

In [5]:
#Note: Modmine sometimes goes down unexpectedly
PATH = '../../data/chip/modENCODE_downloads/'
DWNLD1 = 'http://intermine.modencode.org/release-33/features.do?type=submission&action=export&format=csv&submission=modENCODE_'
DWNLD2 = '&feature=BindingSite'
os.makedirs(PATH, exist_ok=True)

done = []
for name in modid_list: 
    if name not in no_data: 
        if name not in done: 
            #try:
            urllib.request.urlretrieve(DWNLD1+name+DWNLD2, PATH+name) 
            done.append(name)
            #except:
                #urllib.request.urlretrieve(DWNLD1+name+'&feature=ProteinBindingSite', PATH+name) 
                #done.append(name)

HTTPError: HTTP Error 503: Service Unavailable

After the download:
- combine files to make one big file
- get as bed file
- do liftover
- intersect w gene bed file (one idea: if it's too hard to get these as beds, could use data from before and merge on location) 
- clean up table
- phantom peak overlap? 

In [277]:
#combine files into one: 
concat=[]
import glob
for fname in glob.glob('../../data/modENCODE_downloads_2ndtry/[0-9]*'):
    name = os.path.splitext(os.path.basename(fname))[0]
    if name not in no_data:
        df = pd.read_csv(fname)
        #df.astype(int)
        #df[2]
        concat.append(df)     

In [278]:
big = pd.concat(concat)
big.to_csv('../../data/modENCODE_downloads_2ndtry/modENCODE_big', sep='\t', na_rep=':.', header=None, index=False)

In [3]:
#get gene file as a bed:
import pybedtools
from pybedtools import BedTool
from pybedtools.featurefuncs import gff2bed
genes = BedTool('../../data/dmel-all-r6.12.gene_only.chr.gff')
genes_bed = genes.each(gff2bed, name_field='ID').saveas()

In [279]:
#make new id column (set to name/4th column)
#make this a bed and add chr to chromosome 
big.columns = ['DB_id','score','chrom','start','end','strand','modENCODE_id','type','name']

In [280]:
big['id'] = ['id='+str(x) for x in range(0, len(big.score))]

In [281]:
big['chrom'] = ['chr'+x for x in big['chrom']]

In [282]:
bigsub = big[['chrom','start','end','id']]
bigsub.to_csv('../../data/modENCODE_downloads_2ndtry/modENCODE_big.bed', sep='\t', header=None, index=False)

In [123]:
#do liftover

In [12]:
#import liftover file, bed intersect
with open('../../data/modENCODE_downloads_2ndtry/modENCODE_big.liftover') as f: 
    peaks_bed = BedTool(f)
    intersect = genes_bed.intersect(peaks_bed, wb=True).saveas()
    intdf = intersect.to_dataframe()
#then merge back on location to get all the info

In [285]:
trim = intdf[['chrom','start','end','name','score','strand','blockCount']]

In [286]:
merge = trim.merge(big, left_on='blockCount', right_on='id', how='left')
merge.head()

,chrom_x,start_x,end_x,name_x,score_x,strand_x,blockCount,DB_id,score_y,chrom_y,start_y,end_y,strand_y,modENCODE_id,type,name_y,id
0,chr2L,7528,8128,FBgn0031208,.,+,id=280047,Trithorax_like_WPP_ChIP_chip.gff_ID000002,51.150000,chr2L,7492,8128,0,modENCODE_3830,strain,yellow cinnabar brown speck,id=280047
1,chr2L,7528,8128,FBgn0031208,.,+,id=280048,Trithorax_like_WPP_ChIP_chip.gff_ID000002,51.150000,chr2L,7492,8128,0,modENCODE_3830,developmental stage,L3 stage larvae,id=280048
2,chr2L,7528,8128,FBgn0031208,.,+,id=280049,Trithorax_like_WPP_ChIP_chip.gff_ID000002,51.150000,chr2L,7492,8128,0,modENCODE_3830,antibody,KW3-Trl-D2,id=280049
3,chr2L,7528,9484,FBgn0031208,.,+,id=383688,Solexa_Pc_14_16_hr_OR_embryo.gff_ID000002,1.986896,chr2L,5038,13342,0,modENCODE_3957,strain,Oregon-R-modENCODE,id=383688
4,chr2L,7528,9484,FBgn0031208,.,+,id=383689,Solexa_Pc_14_16_hr_OR_embryo.gff_ID000002,1.986896,chr2L,5038,13342,0,modENCODE_3957,developmental stage,Embryo 14-16hr OR,id=383689


In [289]:
another = merge[['chrom_x', 'start_x', 'end_x', 'name_x', 'strand_x', 'blockCount', 'DB_id', 'score_y','modENCODE_id', 'type', 'name_y']]
another.columns=['chrom','start','end','gene','strand','id','DB_id','score','modENCODE_id','type','name']      

In [290]:
len(another.modENCODE_id.unique())

150

In [314]:
another = another.fillna('.')

In [315]:
happy = another.set_index(
    ['modENCODE_id','DB_id','score','chrom','start','end','strand','gene','type']).pivot_table(
        columns='type', index=['DB_id','score','chrom','start','end','strand','modENCODE_id','gene'], 
        values='name',aggfunc='first')

In [316]:
happy = happy.reset_index()

In [317]:
happy['target gene'].unique()

array([None, 'Eip74EF', 'Hr78', 'Stat92E', 'usp', 'lola'], dtype=object)

In [318]:
len(happy['modENCODE_id'].unique())

150

In [319]:
happy.shape

(1415663, 15)

In [29]:
#parse before GFP from DB_id: 
hopefully_genes = []
for item in happy.DB_id:
    new = item.split('_GFP')[0]
    new = new.split('.gff')[0]
    new = new.split('.')[0]
    hopefully_genes.append(new)

In [30]:
len(set(hopefully_genes))

105

In [32]:
# The following two lines will be needed in every python script:
from intermine.webservice import Service
service = Service("http://intermine.modencode.org/release-33/service")

# Get a new query on the class (table) you will be querying:
query = service.new_query("Submission")

# The view specifies the output columns
query.add_view("DCCid", "assayFactor")

# Uncomment and edit the line below (the default) to select a custom sort order:
# query.add_sort_order("Submission.DCCid", "ASC")

# You can edit the constraint values below
query.add_constraint("experimentType", "=", "ChIP-seq", code = "A")
query.add_constraint("organism.commonName", "=", "fruit fly", code = "B")
#query.add_constraint("lab.name", "=", "Kevin White", code = "C")

# Uncomment and edit the code below to specify your own custom logic:
# query.set_logic("A and B and C")
table = []
for row in query.rows():
    newline = [row["DCCid"], row["assayFactor"]]
    table.append(newline)

In [323]:
gene_map = pd.DataFrame(table)
gene_map.columns = ['modENCODE_id','gene']
gene_map.head()

,modENCODE_id,gene
0,modENCODE_2626,Caudal
1,modENCODE_2627,HDAC4a
2,modENCODE_2629,HDAC11
3,modENCODE_2630,HDAC11
4,modENCODE_2633,HDAC3


In [326]:
withmap = happy.merge(gene_map, on='modENCODE_id', how='left')
withmap.head()

,DB_id,score,chrom,start,end,strand,modENCODE_id,gene_x,antibody,cell line,developmental stage,strain,target gene,threshold,tissue,gene_y
0,10T_E16_24h_GFP.1,.,chr2L,516190,516870,+,modENCODE_3229,FBgn0003963,No Antibody Control,None,Embryo 16-24 h,FlyStrain:10T-DFD-GFP:KW:1&oldid=39548,None,None,None,eGFP
1,10T_E16_24h_GFP.10,.,chr2L,3124123,3125019,-,modENCODE_3229,FBgn0015600,No Antibody Control,None,Embryo 16-24 h,FlyStrain:10T-DFD-GFP:KW:1&oldid=39548,None,None,None,eGFP
2,10T_E16_24h_GFP.100,.,chr2L,21306821,21307495,-,modENCODE_3229,FBgn0032940,No Antibody Control,None,Embryo 16-24 h,FlyStrain:10T-DFD-GFP:KW:1&oldid=39548,None,None,None,eGFP
3,10T_E16_24h_GFP.11,.,chr2L,3301741,3302706,-,modENCODE_3229,FBgn0031516,No Antibody Control,None,Embryo 16-24 h,FlyStrain:10T-DFD-GFP:KW:1&oldid=39548,None,None,None,eGFP
4,10T_E16_24h_GFP.113,.,chr2R,1734511,1735276,+,modENCODE_3229,FBgn0087011,No Antibody Control,None,Embryo 16-24 h,FlyStrain:10T-DFD-GFP:KW:1&oldid=39548,None,None,None,eGFP


In [327]:
withmap.columns = ['DB_id', 'score', 'chrom', 'start', 'end', 'strand', 'modENCODE_id',
       'gene', 'antibody', 'cell line', 'developmental stage', 'strain',
       'target gene', 'threshold', 'tissue', 'TF']

In [359]:
#add column for motif_FBgn
symbolmap = pd.read_table('/data/LCDB/lcdb-references/dmel/r6-11/gtf/dmel_r6-11.SYMBOL.csv', sep=',') 
addfbgn = withmap.merge(symbolmap, left_on='TF', right_on='SYMBOL', how='left')
addfbgn = addfbgn.rename(columns={'ENSEMBL': 'motif_FBgn'})

In [360]:
addfbgn = addfbgn[['DB_id', 'score', 'chrom', 'start', 'end', 'strand', 'modENCODE_id',
       'gene', 'antibody', 'cell line', 'developmental stage', 'strain',
       'target gene','tissue', 'TF', 'motif_FBgn']]

In [330]:
symbolmap2 = pd.read_table('../../data/fb_synonym.tsv', sep=' ', header=None)
symbolmap2.columns = ['gene','a','b']
symbolmap2.head()

,gene,a,b
0,FBgn0262029,d,dachs
1,FBgn0052532,CG32532,"CG14203,CG14202,CG14201"
2,FBgn0023536,CG3156,EG:171D11.2
3,FBgn0029718,mRpL30,mitochondrial
4,FBgn0031101,CG1631,NaN


In [331]:
for val in set(addfbgn.TF):
    if val not in list(symbolmap2.a):
        if val not in list(symbolmap2.b):
            print(val)
    

Smc3
dMi-2
Caudal
RNA polymerase II CTD repeat YSPTSPS (phospho S5)
RNA polymerase II
MOF
JMJD2A/KDM4A
dRING
KNI
PCL
Chriz
HP1a
LSD1
NURF301
Huckebein
dSFMBT
POF
RNA polymerase II CTD repeat YSPTSPS (phospho S2)
JHDM1
DLL
CP190
Su(Hw)
dm
RPD3
eGFP
GROUCHO
Yki
CG16778
GAF
HP2
ZW5
HDAC4a


Change all of the incorrect gene names so that symbolmap2 can recognize them: 

In [361]:
addfbgn_copy = addfbgn.copy()

In [362]:
addfbgn_copy.loc[addfbgn.TF == 'dMi-2','TF'] = 'Mi-2'
addfbgn_copy.loc[addfbgn.TF == 'Smc3','TF'] = 'SMC3'
addfbgn_copy.loc[addfbgn.TF == 'Caudal','TF'] = 'caudal'
addfbgn_copy.loc[addfbgn.TF == 'MOF','TF'] = 'mof'
addfbgn_copy.loc[addfbgn.TF == 'JMJD2A/KDM4A','TF'] = 'Kdm4A'
addfbgn_copy.loc[addfbgn.TF == 'dRING','TF'] = 'Sce'
addfbgn_copy.loc[addfbgn.TF == 'Chriz','TF'] = 'Chro'
addfbgn_copy.loc[addfbgn.TF == 'HP1a','TF'] = 'Su(var)205'
addfbgn_copy.loc[addfbgn.TF == 'LSD1','TF'] = 'Su(var)3-3'
addfbgn_copy.loc[addfbgn.TF == 'POF','TF'] = 'Pof'
addfbgn_copy.loc[addfbgn.TF == 'DLL','TF'] = 'Dll'
addfbgn_copy.loc[addfbgn.TF == 'CP190','TF'] = 'Cp190'
addfbgn_copy.loc[addfbgn.TF == 'GROUCHO','TF'] = 'groucho'
addfbgn_copy.loc[addfbgn.TF == 'Yki','TF'] = 'yki'
addfbgn_copy.loc[addfbgn.TF == 'GAF','TF'] = 'Trl'
addfbgn_copy.loc[addfbgn.TF == 'RPD3','TF'] = 'HDAC1'
addfbgn_copy.loc[addfbgn.TF == 'HP2','TF'] = 'Su(var)2-HP2'
addfbgn_copy.loc[addfbgn.TF == 'Su(Hw)','TF'] = 'su(Hw)'
addfbgn_copy.loc[addfbgn.TF == 'NURF301','TF'] = 'E(bx)'
addfbgn_copy.loc[addfbgn.TF == 'dSFMBT','TF'] = 'Sfmbt'
addfbgn_copy.loc[addfbgn.TF == 'ZW5','TF'] = 'dwg'
addfbgn_copy.loc[addfbgn.TF == 'JHDM1','TF'] = 'Kdm2'
addfbgn_copy.loc[addfbgn.TF == 'CG16778','TF'] = 'lov'
addfbgn_copy.loc[addfbgn.TF == 'dm','TF'] = 'Myc'
addfbgn_copy.loc[addfbgn.TF == 'KNI','TF'] = 'kni'
addfbgn_copy.loc[addfbgn.TF == 'PCL','TF'] = 'Pcl'
addfbgn_copy.loc[addfbgn.TF == 'Huckebein','TF'] = 'hkb'
addfbgn_copy.loc[addfbgn.TF == 'HDAC4a','TF'] = 'HDAC4'

In [363]:
for val in set(addfbgn_copy.TF):
    if val not in list(symbolmap2.a):
        if val not in list(symbolmap2.b):
            print(val)

RNA polymerase II CTD repeat YSPTSPS (phospho S5)
RNA polymerase II
RNA polymerase II CTD repeat YSPTSPS (phospho S2)
eGFP


In [364]:
merge2 = addfbgn_copy.merge(symbolmap2, left_on='TF', right_on='a', how='left').merge(symbolmap2, left_on='TF', right_on='b', how='left')

In [365]:
merge2.fillna('')
merge2['motif_FBgn_sum']= merge2.motif_FBgn.combine_first(merge2.gene).combine_first(merge2.gene_y)

In [366]:
almost = merge2[['DB_id', 'score', 'chrom', 'start', 'end', 'strand', 'modENCODE_id',
       'gene_x', 'antibody', 'cell line', 'developmental stage', 'strain',
       'target gene', 'tissue', 'TF', 'motif_FBgn_sum']]

In [367]:
len(almost.motif_FBgn_sum.unique())

72

In [368]:
almost = almost.rename(columns={'gene_x': 'peak_location_fbgn'})
almost = almost.rename(columns={'motif_FBgn_sum': 'TF_fbgn'})

In [369]:
#merge to add column for peak_location_symbol
almost1 = almost.merge(symbolmap, left_on='peak_location_fbgn', right_on='ENSEMBL', how='left')

In [370]:
almost1 = almost1.rename(columns={'SYMBOL': 'peak_location_symbol'})
almost1.columns

Index(['DB_id', 'score', 'chrom', 'start', 'end', 'strand', 'modENCODE_id',
       'peak_location_fbgn', 'antibody', 'cell line', 'developmental stage',
       'strain', 'target gene', 'tissue', 'TF', 'TF_fbgn', 'ENSEMBL',
       'peak_location_symbol'],
      dtype='object')

In [371]:
final = almost1[['DB_id', 'score', 'chrom', 'start', 'end', 'strand', 'modENCODE_id',
       'peak_location_fbgn',
       'peak_location_symbol', 'antibody', 'cell line', 'developmental stage',
       'strain', 'target gene','tissue', 'TF', 'TF_fbgn']]

In [372]:
final.head()

,DB_id,score,chrom,start,end,strand,modENCODE_id,peak_location_fbgn,peak_location_symbol,antibody,cell line,developmental stage,strain,target gene,tissue,TF,TF_fbgn
0,10T_E16_24h_GFP.1,.,chr2L,516190,516870,+,modENCODE_3229,FBgn0003963,ush,No Antibody Control,None,Embryo 16-24 h,FlyStrain:10T-DFD-GFP:KW:1&oldid=39548,None,None,eGFP,NaN
1,10T_E16_24h_GFP.10,.,chr2L,3124123,3125019,-,modENCODE_3229,FBgn0015600,toc,No Antibody Control,None,Embryo 16-24 h,FlyStrain:10T-DFD-GFP:KW:1&oldid=39548,None,None,eGFP,NaN
2,10T_E16_24h_GFP.100,.,chr2L,21306821,21307495,-,modENCODE_3229,FBgn0032940,Mio,No Antibody Control,None,Embryo 16-24 h,FlyStrain:10T-DFD-GFP:KW:1&oldid=39548,None,None,eGFP,NaN
3,10T_E16_24h_GFP.11,.,chr2L,3301741,3302706,-,modENCODE_3229,FBgn0031516,CG9663,No Antibody Control,None,Embryo 16-24 h,FlyStrain:10T-DFD-GFP:KW:1&oldid=39548,None,None,eGFP,NaN
4,10T_E16_24h_GFP.113,.,chr2R,1734511,1735276,+,modENCODE_3229,FBgn0087011,CG41520,No Antibody Control,None,Embryo 16-24 h,FlyStrain:10T-DFD-GFP:KW:1&oldid=39548,None,None,eGFP,NaN


In [373]:
final.to_csv('../../data/modENCODE_downloads_2ndtry/modENCODE_finaltable', sep='\t', header=None, index=False)

In [374]:
final.tissue.unique()

array([None, 'Heads OR'], dtype=object)

In [381]:
len(final['developmental stage'].unique())

27

In [376]:
final['cell line'].unique()

array([None, 'CME W1 Cl.8+', 'Kc167', 'S2-DRSC'], dtype=object)

In [380]:
len(final.strain.unique())

16

In [378]:
final[['developmental stage', 'modENCODE_id']].drop_duplicates().groupby('developmental stage').describe()

modENCODE_id                            
                                   count unique             top freq
developmental stage                                                 
3rd Instar Larvae                     13     13  modENCODE_5121    1
Adult Female                           3      3  modENCODE_3399    1
Adult Male                             1      1   modENCODE_863    1
Embryo 0-12 h                         13     13  modENCODE_3396    1
Embryo 0-4 h                           3      3   modENCODE_854    1
Embryo 0-8 h                          10     10  modENCODE_4074    1
Embryo 1-6 h                           2      2  modENCODE_3400    1
Embryo 12-16 h                         2      2   modENCODE_856    1
Embryo 12-24 h                         2      2  modENCODE_5014    1
Embryo 14-16hr OR                     25     25  modENCODE_5264    1
Embryo 16-20 h                         2      2   modENCODE_857    1
Embryo 16-24 h                        11     11  modENCODE_3390    1
Embryo 20-24 h                         2      2   modENCODE_858    1
Embryo 4-8 h                           2      2   modENCODE_846    1
Embryo 8-12 h                          2      2   modENCODE_985    1
Embryo 8-16 h                         11     11  modENCODE_3402    1
L1 stage larvae                        2      2   modENCODE_859    1
L2 stage larvae                        1      1   modENCODE_984    1
L3 stage larvae                        8      8  modENCODE_3403    1
Mixed Adults 22 days AEL              12     12  modENCODE_5594    1
Pupae                                  2      2   modENCODE_853    1
White prepupae (WPP)                   7      7  modENCODE_4981    1
White prepupae (WPP) + 4-5h            1      1  modENCODE_3398    1
White prepupae (WPP) 10-11             1      1  modENCODE_2641    1
White prepupae (WPP) 30-33h            1      1  modENCODE_2642    1
White prepupae (WPP)+12                1      1  modENCODE_4113    1

In [379]:
final[['cell line', 'modENCODE_id']].drop_duplicates().groupby('cell line').describe()

modENCODE_id                            
                    count unique             top freq
cell line                                            
CME W1 Cl.8+            3      3  modENCODE_5576    1
Kc167                   3      3  modENCODE_5570    1
S2-DRSC                 2      2  modENCODE_2638    1